# Cross-Encoder model

## Data loading

In [1]:
from datasets import load_from_disk
from src.utils.config_management import CONFIG

In [2]:
hf_dataset = load_from_disk(CONFIG['paths']['data']['dalip_hf_dataset'])

In [3]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 'question_view_count', 'question_body', 'question_last_edit_date', 'question_last_activity_date', 'question_title', 'question_tags', 'question_answer_count', 'question_comment_count', 'question_favorite_count', 'question_closed_date', 'question_community_owned_date', 'answer_accepted'],
        num_rows: 42700
    })
    test: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 

## Data preprocessing

In [4]:
from src.utils.text_preprocessing import Preprocessor

In [5]:
preprocessor = Preprocessor(preserve_html_tags=['code'])

In [6]:
hf_dataset = hf_dataset.map(preprocessor, batched=True)

In [7]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 'question_view_count', 'question_body', 'question_last_edit_date', 'question_last_activity_date', 'question_title', 'question_tags', 'question_answer_count', 'question_comment_count', 'question_favorite_count', 'question_closed_date', 'question_community_owned_date', 'answer_accepted', 'question_text', 'answer_text'],
        num_rows: 42700
    })
    test: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_cr

## Fine-tuning

In [8]:
import torch
import os
import math

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

TARGET_COL = 'answer_normalized_score'
MODEL_PATH = 'mmukh/SOBertLarge'
MODEL_NAME = MODEL_PATH.split('/')[-1]
MAX_LENGTH = 2048
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = math.ceil(4 / BATCH_SIZE)
LOSS = 'MSE'
MODEL_OUTPUT_PATH = os.path.join(CONFIG['paths']['models']['dalip_cross-encoder'],
                                 f'cross-encoder_{MODEL_NAME}')

### Define model and tokenize dataset

In [10]:
# import transformers
# transformers.logging.set_verbosity_error()

In [11]:
import torch.nn as nn
from transformers import (MegatronBertForSequenceClassification, PreTrainedTokenizerFast,
                          LongformerForSequenceClassification, LongformerTokenizer,
                          # ModernBertForSequenceClassification, AutoTokenizer
                          )

2025-05-22 15:53:27.602415: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 15:53:27.672706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-22 15:53:27.672744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-22 15:53:27.675700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 15:53:27.690307: I tensorflow/core/platform/cpu_feature_guar

In [12]:
model = MegatronBertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=1).to(device)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at mmukh/SOBertLarge and are newly initialized: ['bert.embeddings.token_type_embeddings.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_PATH)
if not tokenizer.pad_token:
    pad_token_id = model.bert.embeddings.word_embeddings.padding_idx
    print(f'Setting pad token id to {pad_token_id}...')
    tokenizer.pad_token_id = pad_token_id
    print(f'Pad token set to {tokenizer.pad_token}')

Setting pad token id to 0...
Pad token set to <unk>


In [14]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['question_text'], examples['answer_text'], padding=False,
                                 truncation='longest_first', max_length=MAX_LENGTH)
    examples.update(tokenized_inputs)

    examples['label'] = [float(label) for label in examples[TARGET_COL]]

    return examples

In [15]:
train_dataset_tokenized = hf_dataset['train'].map(tokenize, batched=True, remove_columns=hf_dataset['train'].column_names)
test_dataset_tokenized = hf_dataset['test'].map(tokenize, batched=True, remove_columns=hf_dataset['test'].column_names)

### Train model

In [16]:
from transformers import Trainer, TrainingArguments
from src.evaluation import RankingEvaluator
import wandb
import pandas as pd

In [17]:
def trainer_loss_fn(outputs, labels, num_items_in_batch=None):
    logits = outputs['logits'].squeeze(dim=-1)

    if LOSS == 'MSE':
        loss_fn = nn.MSELoss()

    loss = loss_fn(logits, labels)

    return loss

In [18]:
test_question_ids = hf_dataset['test']['question_id']

evaluator = RankingEvaluator(ndcg_k=list(range(1, 11)),
                             ndcg_gain_func='exponential', ndcg_discount_func='logarithmic')

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.squeeze(-1)
    labels = eval_pred.label_ids

    metrics = evaluator(labels, predictions, test_question_ids)
    if TARGET_COL == 'answer_log_normalized_score':
        metrics.pop('mae')

    predictions_df = pd.DataFrame()
    predictions_df['answer_id'] = hf_dataset['test']['answer_id']
    predictions_df[TARGET_COL] = hf_dataset['test'][TARGET_COL]
    predictions_df['predicted_score'] = predictions

    wandb.log({'predictions_table': wandb.Table(dataframe=predictions_df)})

    return metrics

In [19]:
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    logging_steps=1,
    eval_steps=int(len(hf_dataset['train']) / BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS / 10),
    eval_strategy = "steps",
    save_strategy = "epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to='wandb',
    remove_unused_columns=False,
    gradient_checkpointing=True,
    optim="adamw_8bit"
)

In [20]:
run = wandb.init(
    project='dalip-stackoverflow-answer-ranking',
    tags=['cross-encoder']
)
# run = wandb.init(project='dalip-stackoverflow-answer-ranking',
#                  id='hs9zlvj3',
#                  resume='must')

wandb.config.update({
    'preprocessing': preprocessor.__dict__,
    'model_name': MODEL_NAME,
    'vectorizer': {
        'vectorization_type': 'embeddings',
        'max_length': MAX_LENGTH
    },
    'loss_fn': LOSS
})

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    processing_class=tokenizer,
    compute_loss_func=trainer_loss_fn,
    compute_metrics=compute_metrics
)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bunnynobugs to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()
run.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
